In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path


In [ ]:
# =========================
# Parâmetros do ajuste
# =========================
n = 9  # grau do polinômio (mesmo do .m)
entrada = Path("../data/processed/remove_pontos.csv")  # nosso CSV (lat, lon, dg_res_calc_mgal)
saida_coef = Path("../data/processed/Xa_coeficientes_polinomiais.txt")

In [ ]:
# =========================
# Leitura dos dados
# =========================
dfp = pd.read_csv(entrada)

dfp

In [ ]:
# Ajuste os nomes conforme seu CSV
lat = dfp["lat"].to_numpy()
lon = dfp["lon"].to_numpy()
AG_res = dfp["dg_res_calc_mgal"].to_numpy()

In [ ]:
# =========================
# Covariância empírica (versão fiel ao MATLAB)
# - Distâncias em graus (lat/lon já em °)
# - smax = 4° como no script
# - ds ~ "densidade espacial média" (mesma fórmula)
# =========================
def spatial_covariance(lon, lat, values, smax_deg=4.0):
    """
    Replica a lógica do MATLAB:
      - Calcula ds a partir de uma fórmula de densidade
      - Faz pares (i,j), acumula produtos para bins de distância
      - Retorna a covariância média por bin e o vetor de distâncias
    Custo O(N^2): se N for muito grande, considere amostrar os pontos.
    """
    N = len(values)
    # d_s conforme o .m
    ds = np.sqrt((2 * np.pi * (smax_deg / 2) ** 2) / N)

    nbins = int(round(smax_deg / ds)) + 2
    covariance = np.zeros(nbins, dtype=float)
    counts = np.zeros(nbins, dtype=int)

    # Duplo loop (como no MATLAB). Para muitos pontos, isso pode ser pesado.
    for i in range(N):
        xi, yi, gi = lon[i], lat[i], values[i]
        # vetoriza o j loop (mais rápido que 2 for)
        dx = xi - lon
        dy = yi - lat
        r = np.sqrt(dx * dx + dy * dy)  # graus
        # ignora j==i
        r[i] = np.inf
        # máscara dentro do raio
        mask = r < smax_deg
        # índice do bin
        ir = np.round(r[mask] / ds).astype(int) + 1
        # acumula
        covariance.put(ir, covariance[ir] + gi * values[mask])
        counts.put(ir, counts[ir] + 1)

    # média por bin
    valid = counts != 0
    covariance[valid] = covariance[valid] / counts[valid]
    covdist = np.arange(nbins) * ds  # em graus
    return covariance[valid], covdist[valid]

covariance, covdist = spatial_covariance(lon, lat, AG_res, smax_deg=4.0)
s = covdist  # em graus (igual ao MATLAB)

In [ ]:
# =========================
# Ajuste polinomial por MMQ (sem inversa explícita)
# covariance = a0 + a1*s + ... + a_n*s^n
# =========================
# Monta Vandermonde com potências crescentes (1, s, s^2, ..., s^n)
A = np.vstack([s**k for k in range(n+1)]).T   # shape: (m amostras, n+1)
Lb = covariance

# Solução MMQ estável
Xa, *_ = np.linalg.lstsq(A, Lb, rcond=None)   # coeficientes a0..an

# Avaliação do polinômio ajustado
covariance_fit = A @ Xa

# R^2
ss_res = np.sum((Lb - covariance_fit) ** 2)
ss_tot = np.sum((Lb - Lb.mean()) ** 2)
R2 = 1 - ss_res / ss_tot if ss_tot > 0 else np.nan
print(f"R^2 = {R2:.6f}")

# =========================
# Plot comparando empírico vs ajustado
# =========================
plt.plot(s, covariance_fit, label="Fitted covariance (polynomial)", linewidth=1.5)
plt.plot(s, Lb, label="Empirical covariance", linewidth=1.5)
plt.grid(True, which="both", alpha=0.3)
plt.legend()
plt.xlabel("Distance (°)")
plt.ylabel("Covariance (mGal^2)")
plt.title(f"Covariance fit (degree n={n}) — R²={R2:.4f}")
plt.show()

# =========================
# Exporta coeficientes como no MATLAB
# (um coef por linha; se preferir CSV, troque o formato)
# =========================
np.savetxt(saida_coef, Xa[np.newaxis, :], fmt="%.10g")
print(f"Coeficientes salvos em: {saida_coef.resolve()}")
